In [1]:
!pip install torchmetrics==0.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 15.6 MB/s 


In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time
import os

import cv2
from torchmetrics import F1Score
import pickle

In [3]:
path = '/kaggle/input/traffic-sign-datasets/'
path_dataset1 = path+'Indian Traffic Signs Prediction(85 classes)/Indian Traffic Signs Prediction(85 classes)/'
path_dataset2 = path+'Persian Traffic Sign Dataset (PTSD)/Persian Traffic Sign Dataset (PTSD)/'
path_dataset3 = path+'Traffic Signs (GTSRB plus 162 custom classes)/Traffic Signs (GTSRB plus 162 custom classes)/Data_images/'

savepath='/kaggle/working/'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path='/content/drive/My Drive/6721 Project dataset/'
savepath = '/content/drive/My Drive/6721 Dataset/'
print(os.listdir(path))

['Indian Traffic Signs Prediction(85 classes)', 'Persian Traffic Sign Dataset (PTSD)', 'Traffic Signs (GTSRB plus 162 custom classes)']


In [6]:
path_dataset1 = path+"Indian Traffic Signs Prediction(85 classes)/"
path_dataset2 = path+"Persian Traffic Sign Dataset (PTSD)/"
path_dataset3 = path+"Traffic Signs (GTSRB plus 162 custom classes)/Data_images/"

**Data Loaders**

---



In [7]:
def load_data(path_train, val_split, path_test, batch_size, input_size):
  
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    transform_train= transforms.Compose([transforms.Resize(input_size),
                                         transforms.ColorJitter(brightness=(0.5,1.2), contrast=0, saturation=0, hue=0),
                                         transforms.RandomHorizontalFlip(),
#                                          transforms.RandomVerticalFlip(),
                                          transforms.RandomRotation(45),
                                        transforms.RandomAdjustSharpness(0.8),
                                          transforms.ToTensor(),
                                          normalize
                                          ])

    transform_test = transforms.Compose([transforms.Resize(input_size),
                                         transforms.ToTensor(), 
                                         normalize])

    data_train = datasets.ImageFolder(root=path_train, transform=transform_train)
    data_test = datasets.ImageFolder(root=path_test, transform=transform_test)
    
    mappings = data_train.class_to_idx

    val_size = int(len(data_train)*val_split)
    train_size = len(data_train) - val_size

    train_dataset, val_dataset = td.random_split(data_train, [train_size, val_size])
    
    data_loader_train = td.DataLoader(train_dataset,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      drop_last=False,
                                      num_workers=0,       
                                      pin_memory=True)
     
    data_loader_val = td.DataLoader(val_dataset,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=False,
                                    num_workers=0) 
      
    data_loader_test = td.DataLoader(data_test,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=False,
                                   num_workers=0)
    
    return data_loader_train, data_loader_test, data_loader_val, mappings

Compute Class Weights

In [8]:
def get_image_size_dist(path):
    t1 = time.time()

    folders = os.listdir(path)
    total = 0

    class_samples = {}

    for folder in folders:
        samples = os.listdir(path+folder)

        class_samples[folder]=len(samples)

        print("Class: {} has samples: {} ".format(folder,len(samples)))
        total+=len(samples)

    print("\nTotal number of samples: ",total)

    print("\nTime taken: {}".format(time.time()-t1))

    return class_samples, total

In [9]:
def get_class_weights(path_train, mappings):

    class_samples, total = get_image_size_dist(path_train)

    class_weights = [0]*len(mappings.keys())

    for key in mappings.keys():
        class_weights[mappings[key]] = 1 - (class_samples[key]/ total)
      
    return class_weights

Training Setup

---



In [10]:
learning_rate = 0.0001
num_epochs = 10

def get_criterion(class_weights):

    criterion = nn.CrossEntropyLoss(weight = class_weights)

    return criterion

In [11]:
def train_model(model, num_epochs, train_loader, criterion, optimizer, savepath, savepath_results, device):

      total_steps = len(train_loader)

      t1 = time.time()

      train_loss=[]
      train_accuracy=[]
      val_accuracy=[]

      for epoch in range(num_epochs):
          for i, data in enumerate(train_loader):
              
              images, labels = data[0].to(device), data[1].to(device)
              
              model.train()

              # Forward pass
              outputs = model(images)
              loss = criterion(outputs, labels)
              
              # Backprop and optimisation
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
              
              # Train accuracy
              total = labels.size(0)
              _, predicted = torch.max(outputs.data, 1)
              correct = (predicted == labels).sum().item()
              
              if (i + 1) % 10 == 0:
                  model.eval() 
                  with torch.no_grad(): 
                      correctv = 0
                      totalv = 0
                      for datav in val_loader:
                          imagesv, labelsv = datav[0].to(device), datav[1].to(device)
                          outputsv = model(imagesv)
                          _, predictedv = torch.max(outputsv.data, 1)
                          totalv += labelsv.size(0)
                          correctv += (predictedv == labelsv).sum().item()
                          
                      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, Validation Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                          (correct / total) * 100,
                          (correctv / totalv) * 100))
                      
                      train_loss.append(loss.item())
                      train_accuracy.append((correct / total) * 100)
                      val_accuracy.append((correctv / totalv) * 100)
                      
                  
      print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

      print("/n/n Saving model at: ",savepath)
      torch.save(model.state_dict(), savepath)

      results = { "training_loss": train_loss, "accuracy": train_accuracy, "val_accuracy": val_accuracy }

      with open(savepath_results, "wb") as fp:
          pickle.dump(results, fp)

      return model, device

In [12]:
def evaluate_model(model, test_loader, device,nclasses):
   
    model.eval() 

    Y=[]
    y=[]

    with torch.no_grad(): 
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            Y.extend(labels)
            y.extend(predicted)

        acc = (correct / total) * 100
        f1 = F1Score(num_classes=nclasses)
        f1score = f1(torch.IntTensor(y),torch.IntTensor(Y))

        print('Model Evaluation Results on {} test samples'.format(total))
        print('Test Accuracy: ', acc)
        print('Test F1 Score: ',f1score)
        

Architecture 1: AlexNet

On dataset 1:

In [ ]:
nclasses=15

In [ ]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [ ]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
AlexNet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
AlexNet_model.to(device)

Device: cuda:0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_60 has samples: 190 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: GAP_IN_MEDIAN has samples: 180 
Class: CROSS_ROAD has samples: 140 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_50 has samples: 200 
Class: HORN_PROHIBITED has samples: 160 
Class: SPEED_LIMIT_40 has samples: 170 
Class: NO_ENTRY has samples: 174 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: SPEED_LIMIT_30 has samples: 238 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.021085023880004883


In [ ]:
model_savepath = savepath+"alexnetmodel_d1.pt"
results_savepath = savepath+"alexnetmodel_d1_results.pkb"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 2.7092, Accuracy: 0.00%, Validation Accuracy: 12.62%
Epoch [1/10], Step [20/66], Loss: 2.7029, Accuracy: 12.50%, Validation Accuracy: 7.84%
Epoch [1/10], Step [30/66], Loss: 2.7039, Accuracy: 9.38%, Validation Accuracy: 7.84%
Epoch [1/10], Step [40/66], Loss: 2.7054, Accuracy: 0.00%, Validation Accuracy: 7.84%
Epoch [1/10], Step [50/66], Loss: 2.6996, Accuracy: 15.62%, Validation Accuracy: 7.84%
Epoch [1/10], Step [60/66], Loss: 2.6864, Accuracy: 3.12%, Validation Accuracy: 7.84%
Epoch [2/10], Step [10/66], Loss: 2.6826, Accuracy: 6.25%, Validation Accuracy: 9.94%
Epoch [2/10], Step [20/66], Loss: 2.7599, Accuracy: 9.38%, Validation Accuracy: 15.68%
Epoch [2/10], Step [30/66], Loss: 2.6999, Accuracy: 9.38%, Validation Accuracy: 7.84%
Epoch [2/10], Step [40/66], Loss: 2.7221, Accuracy: 9.38%, Validation Accuracy: 11.28%
Epoch [2/10], Step [50/66], Loss: 2.6709, Accuracy: 21.88%, Validation Accuracy: 17.40%
Epoch [2/10], Step [60/66], Loss: 2.6437, Accur

In [ ]:
# for loading model
# trained_AlexNet_model.load_state_dict(torch.load(path+"alexnetmodel.pt"))

In [ ]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  46.901408450704224
Test F1 Score:  tensor(0.4690)


On dataset 2:

In [ ]:
nclasses=12

In [ ]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [ ]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset2+"train/"
path_test = path_dataset2+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
AlexNet_model.to(device)

Device: cuda:0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: Pedestrain has samples: 1014 
Class: NO Stopping has samples: 1075 
Class: Right Turn Prohibited has samples: 436 
Class: Danger has samples: 399 
Class: Compulsory Keep BothSide has samples: 573 
Class: Maximum Speed 30 has samples: 477 
Class: Right Margin has samples: 506 
Class: Cycle crossing has samples: 662 
Class: Roundabouts has samples: 497 
Class: One way Traffic has samples: 684 
Class: Compulsory Keep Right has samples: 703 
Class: No Entry has samples: 930 

Total number of samples:  7956

Time taken: 0.01659393310546875


In [ ]:
model_savepath = savepath+"alexnetmodel_d2.pt"
results_savepath = savepath+"alexnetmodel_d2_results.pkb"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/199], Loss: 2.4780, Accuracy: 12.50%, Validation Accuracy: 12.95%
Epoch [1/10], Step [20/199], Loss: 2.4724, Accuracy: 25.00%, Validation Accuracy: 15.15%
Epoch [1/10], Step [30/199], Loss: 2.4754, Accuracy: 9.38%, Validation Accuracy: 13.45%
Epoch [1/10], Step [40/199], Loss: 2.4546, Accuracy: 18.75%, Validation Accuracy: 13.45%
Epoch [1/10], Step [50/199], Loss: 2.4784, Accuracy: 9.38%, Validation Accuracy: 13.45%
Epoch [1/10], Step [60/199], Loss: 2.3669, Accuracy: 12.50%, Validation Accuracy: 14.39%
Epoch [1/10], Step [70/199], Loss: 2.3794, Accuracy: 21.88%, Validation Accuracy: 29.42%
Epoch [1/10], Step [80/199], Loss: 2.2294, Accuracy: 21.88%, Validation Accuracy: 23.19%
Epoch [1/10], Step [90/199], Loss: 2.0353, Accuracy: 40.62%, Validation Accuracy: 25.46%
Epoch [1/10], Step [100/199], Loss: 2.1142, Accuracy: 21.88%, Validation Accuracy: 26.34%
Epoch [1/10], Step [110/199], Loss: 1.8558, Accuracy: 31.25%, Validation Accuracy: 23.07%
Epoch [1/10], Step [1

In [ ]:
# for loading model
# trained_AlexNet_model.load_state_dict(torch.load(path+"alexnetmodel.pt"))

In [ ]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Model Evaluation Results on 1228 test samples
Test Accuracy:  86.72638436482086
Test F1 Score:  tensor(0.8673)


On dataset 3:

In [ ]:
nclasses=8

In [ ]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [ ]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"

val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
AlexNet_model.to(device)

Device: cuda:0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: 7 has samples: 1433 
Class: 2 has samples: 2243 
Class: 5 has samples: 1853 
Class: 8 has samples: 1403 
Class: 3 has samples: 1403 
Class: 1 has samples: 2213 
Class: 4 has samples: 1973 
Class: 9 has samples: 1463 

Total number of samples:  13984

Time taken: 0.01367330551147461


In [ ]:
model_savepath = savepath+"alexnetmodel_d3.pt"
results_savepath = savepath+"alexnetmodel_d3_results.pkb"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/350], Loss: 2.0765, Accuracy: 25.00%, Validation Accuracy: 15.16%
Epoch [1/10], Step [20/350], Loss: 2.0746, Accuracy: 25.00%, Validation Accuracy: 15.27%
Epoch [1/10], Step [30/350], Loss: 2.0808, Accuracy: 12.50%, Validation Accuracy: 15.27%
Epoch [1/10], Step [40/350], Loss: 2.0645, Accuracy: 18.75%, Validation Accuracy: 15.27%
Epoch [1/10], Step [50/350], Loss: 2.0569, Accuracy: 18.75%, Validation Accuracy: 15.27%
Epoch [1/10], Step [60/350], Loss: 1.9421, Accuracy: 12.50%, Validation Accuracy: 15.27%
Epoch [1/10], Step [70/350], Loss: 2.0887, Accuracy: 0.00%, Validation Accuracy: 17.74%
Epoch [1/10], Step [80/350], Loss: 1.9824, Accuracy: 25.00%, Validation Accuracy: 19.17%
Epoch [1/10], Step [90/350], Loss: 1.9386, Accuracy: 18.75%, Validation Accuracy: 19.03%
Epoch [1/10], Step [100/350], Loss: 2.0648, Accuracy: 15.62%, Validation Accuracy: 17.81%
Epoch [1/10], Step [110/350], Loss: 1.9490, Accuracy: 15.62%, Validation Accuracy: 20.74%
Epoch [1/10], Step [

In [ ]:
# for loading model
# trained_AlexNet_model.load_state_dict(torch.load(path+"alexnetmodel.pt"))

In [ ]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Model Evaluation Results on 4590 test samples
Test Accuracy:  75.14161220043573
Test F1 Score:  tensor(0.7514)


Architecture 2: VGG-11

on dataset 1:

In [ ]:
nclasses=15

In [ ]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=False)

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip


In [ ]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
VGG11_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
VGG11_model.to(device)

Device: cuda:0


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_60 has samples: 190 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: GAP_IN_MEDIAN has samples: 180 
Class: CROSS_ROAD has samples: 140 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_50 has samples: 200 
Class: HORN_PROHIBITED has samples: 160 
Class: SPEED_LIMIT_40 has samples: 170 
Class: NO_ENTRY has samples: 174 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: SPEED_LIMIT_30 has samples: 238 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.021878480911254883


In [ ]:
model_savepath = savepath+"vgg11model_d1.pt"
results_savepath = savepath+"vgg11model_d1_results.pkb"

trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 2.6943, Accuracy: 3.12%, Validation Accuracy: 13.77%
Epoch [1/10], Step [20/66], Loss: 2.7217, Accuracy: 3.12%, Validation Accuracy: 11.85%
Epoch [1/10], Step [30/66], Loss: 2.6502, Accuracy: 12.50%, Validation Accuracy: 11.85%
Epoch [1/10], Step [40/66], Loss: 2.6552, Accuracy: 9.38%, Validation Accuracy: 17.21%
Epoch [1/10], Step [50/66], Loss: 2.6887, Accuracy: 3.12%, Validation Accuracy: 13.58%
Epoch [1/10], Step [60/66], Loss: 2.6066, Accuracy: 18.75%, Validation Accuracy: 18.55%
Epoch [2/10], Step [10/66], Loss: 2.6319, Accuracy: 12.50%, Validation Accuracy: 18.93%
Epoch [2/10], Step [20/66], Loss: 2.6699, Accuracy: 15.62%, Validation Accuracy: 23.33%
Epoch [2/10], Step [30/66], Loss: 2.6797, Accuracy: 15.62%, Validation Accuracy: 19.12%
Epoch [2/10], Step [40/66], Loss: 2.6766, Accuracy: 9.38%, Validation Accuracy: 22.94%
Epoch [2/10], Step [50/66], Loss: 2.4174, Accuracy: 28.12%, Validation Accuracy: 21.61%
Epoch [2/10], Step [60/66], Loss: 2.3

In [ ]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  57.32394366197183
Test F1 Score:  tensor(0.5732)


on dataset 2:

In [ ]:
nclasses=12

In [ ]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset2+"train/"
path_test = path_dataset2+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
VGG11_model.to(device)

Device: cuda:0


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: Pedestrain has samples: 1014 
Class: NO Stopping has samples: 1075 
Class: Right Turn Prohibited has samples: 436 
Class: Danger has samples: 399 
Class: Compulsory Keep BothSide has samples: 573 
Class: Maximum Speed 30 has samples: 477 
Class: Right Margin has samples: 506 
Class: Cycle crossing has samples: 662 
Class: Roundabouts has samples: 497 
Class: One way Traffic has samples: 684 
Class: Compulsory Keep Right has samples: 703 
Class: No Entry has samples: 930 

Total number of samples:  7956

Time taken: 0.018162250518798828


In [ ]:
model_savepath = savepath+"vgg11model_d2.pt"
results_savepath = savepath+"vgg11model_d2_results.pkb"

trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/199], Loss: 2.4560, Accuracy: 3.12%, Validation Accuracy: 13.26%
Epoch [1/10], Step [20/199], Loss: 2.4293, Accuracy: 31.25%, Validation Accuracy: 19.99%
Epoch [1/10], Step [30/199], Loss: 2.4835, Accuracy: 25.00%, Validation Accuracy: 18.98%
Epoch [1/10], Step [40/199], Loss: 2.4009, Accuracy: 15.62%, Validation Accuracy: 21.18%
Epoch [1/10], Step [50/199], Loss: 2.2865, Accuracy: 34.38%, Validation Accuracy: 18.73%
Epoch [1/10], Step [60/199], Loss: 2.1977, Accuracy: 31.25%, Validation Accuracy: 24.26%
Epoch [1/10], Step [70/199], Loss: 2.3163, Accuracy: 28.12%, Validation Accuracy: 29.48%
Epoch [1/10], Step [80/199], Loss: 2.1409, Accuracy: 31.25%, Validation Accuracy: 30.86%
Epoch [1/10], Step [90/199], Loss: 2.1579, Accuracy: 15.62%, Validation Accuracy: 28.72%
Epoch [1/10], Step [100/199], Loss: 1.6897, Accuracy: 43.75%, Validation Accuracy: 30.99%
Epoch [1/10], Step [110/199], Loss: 1.7459, Accuracy: 43.75%, Validation Accuracy: 36.02%
Epoch [1/10], Step [

In [ ]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

Model Evaluation Results on 1228 test samples
Test Accuracy:  85.26058631921825
Test F1 Score:  tensor(0.8526)


on dataset 3:

In [ ]:
nclasses=8

In [ ]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=False)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
VGG11_model.to(device)

Device: cuda:0


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: 1 has samples: 2213 
Class: 5 has samples: 1853 
Class: 7 has samples: 1433 
Class: 2 has samples: 2243 
Class: 3 has samples: 1413 
Class: 4 has samples: 1973 
Class: 9 has samples: 1463 
Class: 8 has samples: 1403 

Total number of samples:  13994

Time taken: 0.17267417907714844


In [ ]:
model_savepath = savepath+"vgg11model_d3.pt"
results_savepath = savepath+"vgg11model_d3_results.pkb"

trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/350], Loss: 2.0205, Accuracy: 25.00%, Validation Accuracy: 19.23%
Epoch [1/10], Step [20/350], Loss: 2.0262, Accuracy: 9.38%, Validation Accuracy: 18.12%
Epoch [1/10], Step [30/350], Loss: 2.0501, Accuracy: 6.25%, Validation Accuracy: 17.80%
Epoch [1/10], Step [40/350], Loss: 1.9865, Accuracy: 28.12%, Validation Accuracy: 21.02%
Epoch [1/10], Step [50/350], Loss: 2.0470, Accuracy: 9.38%, Validation Accuracy: 21.27%
Epoch [1/10], Step [60/350], Loss: 2.0915, Accuracy: 9.38%, Validation Accuracy: 20.34%
Epoch [1/10], Step [70/350], Loss: 1.9894, Accuracy: 18.75%, Validation Accuracy: 21.12%
Epoch [1/10], Step [80/350], Loss: 1.9589, Accuracy: 31.25%, Validation Accuracy: 21.62%
Epoch [1/10], Step [90/350], Loss: 1.9784, Accuracy: 9.38%, Validation Accuracy: 21.48%
Epoch [1/10], Step [100/350], Loss: 1.9043, Accuracy: 25.00%, Validation Accuracy: 19.87%
Epoch [1/10], Step [110/350], Loss: 2.0611, Accuracy: 21.88%, Validation Accuracy: 20.01%
Epoch [1/10], Step [120/

In [ ]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

Model Evaluation Results on 4590 test samples
Test Accuracy:  92.94117647058823
Test F1 Score:  tensor(0.9294)


Architecture 3: ResNet-18

on dataset 1:

In [ ]:
nclasses=15

In [ ]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [ ]:
ResNet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
ResNet_model.to(device)

Device: cuda:0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_60 has samples: 190 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: GAP_IN_MEDIAN has samples: 180 
Class: CROSS_ROAD has samples: 140 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_50 has samples: 200 
Class: HORN_PROHIBITED has samples: 160 
Class: SPEED_LIMIT_40 has samples: 170 
Class: NO_ENTRY has samples: 174 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: SPEED_LIMIT_30 has samples: 238 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.0189816951751709


In [ ]:
model_savepath = savepath+"resnetmodel_d1.pt"
results_savepath = savepath+"resnetmodel_d1_results.pkb"

trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 2.6229, Accuracy: 9.38%, Validation Accuracy: 3.82%
Epoch [1/10], Step [20/66], Loss: 2.8387, Accuracy: 6.25%, Validation Accuracy: 8.80%
Epoch [1/10], Step [30/66], Loss: 2.5998, Accuracy: 9.38%, Validation Accuracy: 13.58%
Epoch [1/10], Step [40/66], Loss: 2.5879, Accuracy: 18.75%, Validation Accuracy: 24.47%
Epoch [1/10], Step [50/66], Loss: 2.4863, Accuracy: 9.38%, Validation Accuracy: 26.96%
Epoch [1/10], Step [60/66], Loss: 2.4780, Accuracy: 21.88%, Validation Accuracy: 28.68%
Epoch [2/10], Step [10/66], Loss: 2.1310, Accuracy: 34.38%, Validation Accuracy: 32.89%
Epoch [2/10], Step [20/66], Loss: 2.1357, Accuracy: 31.25%, Validation Accuracy: 36.14%
Epoch [2/10], Step [30/66], Loss: 2.3241, Accuracy: 40.62%, Validation Accuracy: 38.81%
Epoch [2/10], Step [40/66], Loss: 2.1849, Accuracy: 21.88%, Validation Accuracy: 38.05%
Epoch [2/10], Step [50/66], Loss: 1.7216, Accuracy: 56.25%, Validation Accuracy: 39.77%
Epoch [2/10], Step [60/66], Loss: 1.78

In [ ]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  53.09859154929577
Test F1 Score:  tensor(0.5310)


on dataset 2:

In [ ]:
nclasses=12

In [ ]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [ ]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset2+"train/"
path_test = path_dataset2+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
ResNet_model.to(device)

Device: cuda:0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: Pedestrain has samples: 1014 
Class: NO Stopping has samples: 1075 
Class: Right Turn Prohibited has samples: 436 
Class: Danger has samples: 399 
Class: Compulsory Keep BothSide has samples: 573 
Class: Maximum Speed 30 has samples: 477 
Class: Right Margin has samples: 506 
Class: Cycle crossing has samples: 662 
Class: Roundabouts has samples: 497 
Class: One way Traffic has samples: 684 
Class: Compulsory Keep Right has samples: 703 
Class: No Entry has samples: 930 

Total number of samples:  7956

Time taken: 0.020913362503051758


In [ ]:
model_savepath = savepath+"resnetmodel_d2.pt"
results_savepath = savepath+"resnetmodel_d2_results.pkb"

trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/199], Loss: 2.5473, Accuracy: 9.38%, Validation Accuracy: 10.69%
Epoch [1/10], Step [20/199], Loss: 2.4314, Accuracy: 12.50%, Validation Accuracy: 11.94%
Epoch [1/10], Step [30/199], Loss: 2.3604, Accuracy: 15.62%, Validation Accuracy: 13.76%
Epoch [1/10], Step [40/199], Loss: 2.2788, Accuracy: 21.88%, Validation Accuracy: 22.56%
Epoch [1/10], Step [50/199], Loss: 2.2715, Accuracy: 12.50%, Validation Accuracy: 24.89%
Epoch [1/10], Step [60/199], Loss: 1.9837, Accuracy: 21.88%, Validation Accuracy: 24.89%
Epoch [1/10], Step [70/199], Loss: 1.9257, Accuracy: 37.50%, Validation Accuracy: 26.02%
Epoch [1/10], Step [80/199], Loss: 1.9212, Accuracy: 34.38%, Validation Accuracy: 30.17%
Epoch [1/10], Step [90/199], Loss: 1.7146, Accuracy: 34.38%, Validation Accuracy: 35.39%
Epoch [1/10], Step [100/199], Loss: 1.7513, Accuracy: 37.50%, Validation Accuracy: 39.60%
Epoch [1/10], Step [110/199], Loss: 1.9002, Accuracy: 40.62%, Validation Accuracy: 43.18%
Epoch [1/10], Step [

In [ ]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

Model Evaluation Results on 1228 test samples
Test Accuracy:  79.64169381107492
Test F1 Score:  tensor(0.7964)


on dataset 3:

In [ ]:
nclasses=8

In [ ]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [ ]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
ResNet_model.to(device)

Device: cuda:0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: 7 has samples: 1433 
Class: 2 has samples: 2243 
Class: 5 has samples: 1853 
Class: 8 has samples: 1403 
Class: 3 has samples: 1403 
Class: 1 has samples: 2213 
Class: 4 has samples: 1973 
Class: 9 has samples: 1463 

Total number of samples:  13984

Time taken: 0.01495051383972168


In [ ]:
model_savepath = savepath+"resnetmodel_d3.pt"
results_savepath = savepath+"resnetmodel_d3_results.pkb"

trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/350], Loss: 2.3874, Accuracy: 12.50%, Validation Accuracy: 12.55%
Epoch [1/10], Step [20/350], Loss: 2.0473, Accuracy: 12.50%, Validation Accuracy: 12.73%
Epoch [1/10], Step [30/350], Loss: 2.1272, Accuracy: 25.00%, Validation Accuracy: 13.20%
Epoch [1/10], Step [40/350], Loss: 2.1378, Accuracy: 12.50%, Validation Accuracy: 16.56%
Epoch [1/10], Step [50/350], Loss: 2.0584, Accuracy: 12.50%, Validation Accuracy: 16.49%
Epoch [1/10], Step [60/350], Loss: 2.1654, Accuracy: 12.50%, Validation Accuracy: 21.67%
Epoch [1/10], Step [70/350], Loss: 1.8626, Accuracy: 37.50%, Validation Accuracy: 20.78%
Epoch [1/10], Step [80/350], Loss: 2.0849, Accuracy: 18.75%, Validation Accuracy: 23.18%
Epoch [1/10], Step [90/350], Loss: 1.9116, Accuracy: 18.75%, Validation Accuracy: 24.03%
Epoch [1/10], Step [100/350], Loss: 1.8461, Accuracy: 25.00%, Validation Accuracy: 23.68%
Epoch [1/10], Step [110/350], Loss: 1.9236, Accuracy: 18.75%, Validation Accuracy: 24.57%
Epoch [1/10], Step 

In [ ]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

Model Evaluation Results on 4590 test samples
Test Accuracy:  61.808278867102395
Test F1 Score:  tensor(0.6181)


Transfer Learning Models:

Alexnet with dataset 1

In [13]:
nclasses=15

In [14]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to /root/.cache/torch/hub/v0.6.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [15]:
# layers = AlexNet_model.state_dict().keys()

for name, param in AlexNet_model.named_parameters():
    if name.startswith('features'):
        param.requires_grad = False

In [16]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [17]:
for name, param in AlexNet_model.named_parameters():
        print("Layer: {} has requires_grad set to: {}".format(name, param.requires_grad))

Layer: features.0.weight has requires_grad set to: False
Layer: features.0.bias has requires_grad set to: False
Layer: features.3.weight has requires_grad set to: False
Layer: features.3.bias has requires_grad set to: False
Layer: features.6.weight has requires_grad set to: False
Layer: features.6.bias has requires_grad set to: False
Layer: features.8.weight has requires_grad set to: False
Layer: features.8.bias has requires_grad set to: False
Layer: features.10.weight has requires_grad set to: False
Layer: features.10.bias has requires_grad set to: False
Layer: classifier.1.weight has requires_grad set to: True
Layer: classifier.1.bias has requires_grad set to: True
Layer: classifier.4.weight has requires_grad set to: True
Layer: classifier.4.bias has requires_grad set to: True
Layer: classifier.6.weight has requires_grad set to: True
Layer: classifier.6.bias has requires_grad set to: True


In [18]:
AlexNet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [19]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [20]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = AlexNet_model.to(device)

Device: cuda:0


In [22]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: CROSS_ROAD has samples: 140 
Class: GAP_IN_MEDIAN has samples: 180 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: HORN_PROHIBITED has samples: 160 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: NO_ENTRY has samples: 174 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_40 has samples: 170 
Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_30 has samples: 238 
Class: SPEED_LIMIT_50 has samples: 200 
Class: SPEED_LIMIT_60 has samples: 190 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.045252084732055664


In [23]:
model_savepath = savepath+"alexnetmodel_tl_d1.pt"
results_savepath = savepath+"alexnetmodel_tl_d1_results.pkb"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 2.6811, Accuracy: 12.50%, Validation Accuracy: 11.47%
Epoch [1/10], Step [20/66], Loss: 2.4506, Accuracy: 28.12%, Validation Accuracy: 23.14%
Epoch [1/10], Step [30/66], Loss: 2.6334, Accuracy: 9.38%, Validation Accuracy: 27.53%
Epoch [1/10], Step [40/66], Loss: 2.4380, Accuracy: 40.62%, Validation Accuracy: 34.80%
Epoch [1/10], Step [50/66], Loss: 2.0842, Accuracy: 34.38%, Validation Accuracy: 44.74%
Epoch [1/10], Step [60/66], Loss: 2.0753, Accuracy: 34.38%, Validation Accuracy: 45.89%
Epoch [2/10], Step [10/66], Loss: 1.9141, Accuracy: 40.62%, Validation Accuracy: 52.58%
Epoch [2/10], Step [20/66], Loss: 1.8802, Accuracy: 34.38%, Validation Accuracy: 53.54%
Epoch [2/10], Step [30/66], Loss: 1.4238, Accuracy: 59.38%, Validation Accuracy: 56.21%
Epoch [2/10], Step [40/66], Loss: 1.7493, Accuracy: 40.62%, Validation Accuracy: 58.51%
Epoch [2/10], Step [50/66], Loss: 1.5129, Accuracy: 59.38%, Validation Accuracy: 62.91%
Epoch [2/10], Step [60/66], Loss:

In [24]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  69.85915492957746
Test F1 Score:  tensor(0.6986)
Model Evaluation Results on 710 test samples
Test Accuracy:  69.85915492957746
Test F1 Score:  tensor(0.6986)


ResNet-18 with dataset 1

In [ ]:
nclasses=15

In [ ]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

In [ ]:
# layers = ResNet_model.state_dict().keys()

for name, param in ResNet_model.named_parameters():
    if not name.startswith('fc'):
        param.requires_grad = False

In [ ]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [ ]:
for name, param in ResNet_model.named_parameters():
        print("Layer: {} has requires_grad set to: {}".format(name, param.requires_grad))

In [ ]:
ResNet_model.eval()

In [ ]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = ResNet_model.to(device)

In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

In [35]:
model_savepath = savepath+"resnetmodel_tl_d1.pt"
results_savepath = savepath+"resnetmodel_tl_d1_results.pkb"

trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 3.2227, Accuracy: 3.12%, Validation Accuracy: 7.84%
Epoch [1/10], Step [20/66], Loss: 2.7741, Accuracy: 6.25%, Validation Accuracy: 8.22%
Epoch [1/10], Step [30/66], Loss: 2.9595, Accuracy: 3.12%, Validation Accuracy: 8.41%
Epoch [1/10], Step [40/66], Loss: 2.8735, Accuracy: 6.25%, Validation Accuracy: 7.84%
Epoch [1/10], Step [50/66], Loss: 2.8367, Accuracy: 3.12%, Validation Accuracy: 7.65%
Epoch [1/10], Step [60/66], Loss: 2.8102, Accuracy: 6.25%, Validation Accuracy: 7.65%
Epoch [2/10], Step [10/66], Loss: 2.8524, Accuracy: 6.25%, Validation Accuracy: 6.50%
Epoch [2/10], Step [20/66], Loss: 2.8355, Accuracy: 6.25%, Validation Accuracy: 9.56%
Epoch [2/10], Step [30/66], Loss: 2.7649, Accuracy: 15.62%, Validation Accuracy: 8.99%
Epoch [2/10], Step [40/66], Loss: 2.7425, Accuracy: 9.38%, Validation Accuracy: 10.52%
Epoch [2/10], Step [50/66], Loss: 2.8655, Accuracy: 9.38%, Validation Accuracy: 9.37%
Epoch [2/10], Step [60/66], Loss: 2.8221, Accuracy: 

In [36]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  26.619718309859152
Test F1 Score:  tensor(0.2662)
